# LangGraph - Calling Tools and Checkpointer

* [Deeplearning.ai - AI Agents in LangGraph](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/l7rgk/langgraph-components)
* [Tavily](https://app.tavily.com/home) - Get API Key here
* [langchain-tavily](https://docs.langchain.com/oss/python/integrations/providers/tavily)

In [1]:
!pip install -U langchain-tavily --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import logging
import os
import operator
import sys
from typing import (
    Annotated, List, Dict, TypedDict, Callable, Union
)
import operator

import sqlite3
from langchain_core.messages import BaseMessage, SystemMessage, ToolMessage
from langgraph.graph import END, StateGraph, START

import openai
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

from IPython.display import Image, display

In [3]:
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logger = logging.getLogger('LOGGER_NAME')

# API Keys

In [4]:
path_to_openai_key:str = os.path.expanduser('~/.openai/api_key')
with open(path_to_openai_key, 'r', encoding='utf-8') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

path_to_tavily_key:str = os.path.expanduser('~/.tavily/api_key')
with open(path_to_tavily_key, 'r', encoding='utf-8') as file:
    os.environ["TAVILY_API_KEY"] = file.read().strip()

# Model

In [5]:
MODEL: str = 'gpt-4.1'

model = init_chat_model(f"openai:{MODEL}")

# Tools

In [6]:
search = TavilySearchResults(max_results=4)

/var/folders/_4/8v285hqs45xfzk0l1nlr3yq40000gn/T/ipykernel_58913/2774767458.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=4)


# LLM Prompt

In [7]:
system_prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

# LangGraph Application

In [9]:
class AgentState(TypedDict):
    """The state of the agent passed between nodes.

    Attributes:
        messages: A list of messages in the conversation, using operator.add
            to append new messages to the existing history.
    """
    messages: Annotated[List[AnyMessage], operator.add]

class Agent:
    def __init__(
        self,
        model,
        tools: List[Callable],
        db_path: str = "checkpoints.db", # Pass path instead of open connection
        system_prompt: str = ""
    ):
        self.system_prompt = system_prompt
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        
        # Initialize connection and saver internally to manage lifecycle
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.checkpointer = SqliteSaver(self.conn)

        # Build Graph
        builder = StateGraph(AgentState)
        builder.add_node("llm", self.call_llm)
        builder.add_node("action", self.take_action)

        builder.add_edge(START, "llm")
        builder.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        builder.add_edge("action", "llm")

        self.graph = builder.compile(checkpointer=self.checkpointer)

    def exists_action(self, state: AgentState) -> bool:
        """Determines if the LLM has requested a tool call.
        Args:
            state: The current graph state.

        Returns:
            True if tool calls are present, False otherwise.
        """
        last_message = state["messages"][-1]
        return len(getattr(last_message, "tool_calls", [])) > 0

    def call_llm(self, state: AgentState) -> Dict[str, List[BaseMessage]]:
        """Invokes the LLM with the current message history.
        Args:
            state: The current graph state.

        Returns:
            A dictionary updating the 'messages' key with the LLM's response.
        """
        messages = state["messages"]
        
        # Add system prompt to the start of the conversation if it exists
        if self.system_prompt and not any(isinstance(m, SystemMessage) for m in messages):
            messages = [SystemMessage(content=self.system_prompt)] + messages
            
        response = self.model.invoke(messages)
        return {"messages": [response]}

    def take_action(self, state: AgentState) -> Dict[str, List[ToolMessage]]:
        """Executes the tool calls requested by the LLM.

        Args:
            state: The current graph state.

        Returns:
            A dictionary containing the ToolMessages with results.
        """
        last_message = state["messages"][-1]
        tool_calls = getattr(last_message, "tool_calls", [])
        results = []

        for t in tool_calls:
            print(f"Calling tool: {t['name']}")
            if t["name"] not in self.tools:
                print("\nbad tool name:[%s]", t["name"])
                result = f"Error: {t['name']} is not a valid tool. Please retry."
            else:
                result = self.tools[t["name"]].invoke(t["args"])

            results.append(
                ToolMessage(
                    tool_call_id=t["id"],
                    name=t["name"],
                    content=str(result)
                )
            )

        print("Action complete. Returning to model.")
        return {"messages": results}

    def close(self):
        """Manually close the database connection."""
        if hasattr(self, 'conn') and self.conn:
            self.conn.close()
            print(f"Database connection to {self.db_path} closed.")

    def __del__(self):
        """Ensures the connection is closed when the object is garbage collected."""
        self.close()

    def __enter__(self):
        """Support for 'with Agent(...) as agent:'"""
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Closes connection automatically when exiting a 'with' block."""
        self.close()

    def __call__(self, queries: List[str], config:Dict):
        """Runs the graph with multiple queries and a provided configuration.
        Args:
            queries: List of user input strings.
            config: Dictionary containing 'configurable' with 'thread_id'.
        """        
        return self.graph.invoke(
            input=AgentState(messages=[HumanMessage(content=query) for query in queries]),
            config=config
        )

# Execution

In [11]:
with Agent(model=model, tools=[search], db_path="checkpoints.db") as app:
    responses = app(
        queries=["What is the weather in NYC?"],
        config={"configurable": {"thread_id": "NYC"}}
    )
# Connection closes automatically here

Calling tool: tavily_search_results_json
Action complete. Returning to model.
Database connection to checkpoints.db closed.


In [13]:
for index, answer in enumerate(responses['messages']):
    print(f"[message:{index:02}] " + '-' * 80)
    print(answer.content + '\n')

[message:00] --------------------------------------------------------------------------------
What is the weather in NYC?

[message:01] --------------------------------------------------------------------------------


[message:02] --------------------------------------------------------------------------------
[{'title': 'New York weather in December 2025 - Weather25.com', 'url': 'https://www.weather25.com/north-america/usa/new-york?page=month&month=December', 'content': 'weather25.com\nSearch\nweather in United States\nRemove from your favorite locations\nAdd to my locations\nShare\nweather in United States\n\n# New York weather in December 2025\n\nClear\nOvercast\nMist\nPartly cloudy\nOvercast\nLight rain shower\nPartly cloudy\nPatchy light snow\nOvercast\nCloudy\nPartly cloudy\nPartly cloudy\nMist\nPartly cloudy\n\n## The average weather in New York in December\n\nThe weather in New York in December is very cold with temperatures between 1°C and 5°C, warm clothes are a must. [...] 

# Pretty Print

In [14]:
import ast
from IPython.display import display, Markdown

raw_data = responses['messages'][-2].content
data = ast.literal_eval(raw_data)

# 2. Build the Markdown Table
table_md = "| Score | Title | URL |\n| :--- | :--- | :--- |\n"

for entry in data:
    # We truncate the title if it's too long and format the link
    short_title = (entry['title'][:75] + '..') if len(entry['title']) > 75 else entry['title']
    link = f"[{short_title}]({entry['url']})"
    score = f"{entry['score']:.2f}" # Format score to 2 decimal places
    
    table_md += f"| {score} | {link} | [Link]({entry['url']}) |\n"

# 3. Render
display(Markdown("## Search Results Summary"))
display(Markdown(table_md))

# 4. Optional: Display detailed content in collapsible sections
display(Markdown("---"))
display(Markdown("### Detailed Source Content"))
for i, entry in enumerate(data):
    detail_md = f"""
<details>
  <summary><b>Source {i+1}: {entry['title']}</b></summary>
  
  {entry['content']}
  
</details>
"""
    display(Markdown(detail_md))

## Search Results Summary

| Score | Title | URL |
| :--- | :--- | :--- |
| 0.87 | [New York weather in December 2025 - Weather25.com](https://www.weather25.com/north-america/usa/new-york?page=month&month=December) | [Link](https://www.weather25.com/north-america/usa/new-york?page=month&month=December) |
| 0.45 | [New York, NY Monthly Weather - AccuWeather](https://www.accuweather.com/en/us/new-york/10021/december-weather/349727) | [Link](https://www.accuweather.com/en/us/new-york/10021/december-weather/349727) |
| 0.44 | [60-Day Extended Weather Forecast for New York, NY | Almanac.com](https://www.almanac.com/weather/longrange/NY/New%20York) | [Link](https://www.almanac.com/weather/longrange/NY/New%20York) |
| 0.22 | [Monthly Weather Forecast for Central Park, Manhattan, New York](https://weather.com/weather/monthly/l/Central+Park+Manhattan+New+York?canonicalCityId=60b5c1800361f33890fcedaa749a5e3a) | [Link](https://weather.com/weather/monthly/l/Central+Park+Manhattan+New+York?canonicalCityId=60b5c1800361f33890fcedaa749a5e3a) |


---

### Detailed Source Content


<details>
  <summary><b>Source 1: New York weather in December 2025 - Weather25.com</b></summary>
  
  weather25.com
Search
weather in United States
Remove from your favorite locations
Add to my locations
Share
weather in United States

# New York weather in December 2025

Clear
Overcast
Mist
Partly cloudy
Overcast
Light rain shower
Partly cloudy
Patchy light snow
Overcast
Cloudy
Partly cloudy
Partly cloudy
Mist
Partly cloudy

## The average weather in New York in December

The weather in New York in December is very cold with temperatures between 1°C and 5°C, warm clothes are a must. [...] | 14 Overcast 4° /-1° | 15 Overcast 3° /-3° | 16 Cloudy 5° /0° | 17 Overcast 6° /0° | 18 Partly cloudy 4° /-2° | 19 Overcast 3° /-4° | 20 Sunny 2° /-4° |
| 21 Sunny 7° /-2° | 22 Overcast 2° /-5° | 23 Mist 1° /-1° | 24 Partly cloudy 5° /-2° | 25 Overcast 2° /-1° | 26 Light rain shower 12° /-1° | 27 Partly cloudy 1° /-4° |
| 28 Patchy light snow 5° /-3° | 29 Overcast -6° /-8° | 30 Cloudy 0° /-8° | 31 Partly cloudy -5° /-8° |  |  |  | [...] | Sun | Mon | Tue | Wed | Thu | Fri | Sat |
 ---  ---  --- 
|  | 1 Overcast 4° /0° | 2 Sunny 6° /0° | 3 Partly cloudy 6° /0° | 4 Overcast 4° /0° | 5 Overcast 4° /-1° | 6 Overcast 5° /-1° |
| 7 Partly cloudy 4° /0° | 8 Overcast 5° /-1° | 9 Partly cloudy 4° /-1° | 10 Moderate rain 8° /1° | 11 Partly cloudy 8° /1° | 12 Sunny 5° /-1° | 13 Sunny 5° /-1° |
  
</details>



<details>
  <summary><b>Source 2: New York, NY Monthly Weather - AccuWeather</b></summary>
  
  # New York, NY

New York

New York

## Around the Globe

Around the Globe

### Hurricane Tracker

### Severe Weather

### Radar & Maps

### News & Features

### Astronomy

### Business

### Climate

### Health

### Recreation

### Sports

### Travel

### Warnings

### Data Suite

### Forensics

### Advertising

### Superior Accuracy™

### Video

### Winter Center

## Monthly

## December

## 2025

## Daily

## Temperature Graph

## Further Ahead

Further Ahead

### January 2026 [...] ### February 2026

### March 2026

## Around the Globe

Around the Globe

### Hurricane Tracker

### Severe Weather

### Radar & Maps

### News

### Video

### Winter Center

Top Stories

Winter Weather

Northeastern US to get a mixed bag of snow, ice and rain this week

6 hours ago

Weather Forecasts

Christmas week US travelers to face stormy weather on West Coast

7 hours ago

Weather News

Largest wildlife overpass in North America opens across 6-lane highway

2 days ago

Severe Weather [...] Is it safe to eat snow? Here's what the science says

4 days ago

Astronomy

Interstellar comet 3I/ATLAS swings by Earth this week

2 days ago

## Weather Near New York:

...

...

...
  
</details>



<details>
  <summary><b>Source 3: 60-Day Extended Weather Forecast for New York, NY | Almanac.com</b></summary>
  
  Almanac Logo
Almanac.com
Almanac.com

# 60-Day Extended Weather Forecast for New York, NY

## Free 2-Month Weather Forecast

December 2025 Long Range Weather Forecast for Atlantic Corridor [...] | Dates | Weather Conditions |
 --- |
| Dec 1-3 | Showers, mild |
| Dec 4-12 | Snow showers north, sunny south; turning cold |
| Dec 13-15 | Sunny, cold |
| Dec 16-29 | Rain and snow showers, then sunny; turning very cold |
| Dec 30-31 | Snowy, cold |
|  |  |
 --- |
| December | temperature 38° (3° below avg.) precipitation 1.5" (2" below avg.) |

January 2026 Long Range Weather Forecast for Atlantic Corridor [...] November 2025 to October 2026

See the complete 12-month weather predictions in The Old Farmer's Almanac.

## Weather Trend Graph

November 2025 to October 2026

Weather Trend Graph for Atlantic Corridor

#### Get Almanac's Daily Updates

BONUS: You'll also receive our free Beginner Gardening Guide!
  
</details>



<details>
  <summary><b>Source 4: Monthly Weather Forecast for Central Park, Manhattan, New York</b></summary>
  
  | Yesterday | 38° | 35° | 0.00 in |
| Last 7 Days | 57° | 18° | 1.31 |
| Month to Date | 57° | 18° | 2.43 |

| Historical Monthly Avg |

| December | 44° | 34° | 4.38 |
| January | 40° | 28° | 3.64 |
| February | 42° | 30° | 3.19 |

## Sun & Moon

## Stay Safe

## Travel Far, Travel Often

The Weather Company
The Weather Channel
Weather Underground
Storm Radar
Georgia Org
Link to eSSENTIAL Accessibility assistive technology [...] We recognize our responsibility to use data and technology for good. We may use or share your data with our data vendors. Take control of your data.

The Weather Channel is the world's most accurate forecaster according to ForecastWatch, Global and Regional Weather Forecast Accuracy Overview, 2021-2024, commissioned by The Weather Company.

Weather Channel

© The Weather Company, LLC 2025 [...] ## Recents

## Weather Forecasts

## Radar & Maps

## News & Media

## Products & Account

## Lifestyle

### Specialty Forecasts

# Monthly Weather- Central Park, Manhattan, New York

## Night

Clear skies. Low 27F. Winds WNW at 10 to 15 mph.

## We Love Our Critters

## Travel

## Home, Garage & Garden

| Dec 21 | High | Low | Precip |
 ---  --- |
| Averages | 43°F | 32° 
| Records | 71°(2013) | 2°(1872) 

| Reported Conditions |
  
</details>
